In [4]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import numpy as np
import torch.nn as nn
import torch
from dataset import ManualFeatureDataModule
from transformer import TransformerClassifierModule

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
data_dir = Path("/media/nvme1/icare-data/6h-combined")
label_dir = Path("/home/bc299/icare/artifacts/patient_data.csv")
dm = ManualFeatureDataModule(root_dir=data_dir, labels_csv=label_dir)
dm.setup()

x, y = next(iter(dm.train_dataloader()))
x.size()

torch.Size([32, 14, 144])

In [6]:
model = TransformerClassifierModule(input_size=14,
                                    d_model=128,
                                    nhead=8,
                                    num_layers=2,
                                    dropout=0.2,
                                    learning_rate=1e-4)
model(x)

tensor([[-1.1421, -0.3844],
        [-1.4543, -0.2660],
        [-1.2453, -0.3395],
        [-0.6685, -0.7184],
        [-0.9676, -0.4780],
        [-1.3185, -0.3113],
        [-0.9360, -0.4979],
        [-1.4586, -0.2647],
        [-1.0151, -0.4500],
        [-1.2229, -0.3487],
        [-1.0543, -0.4284],
        [-0.9811, -0.4699],
        [-0.7210, -0.6660],
        [-0.5947, -0.8023],
        [-1.4542, -0.2660],
        [-1.1743, -0.3697],
        [-0.8298, -0.5729],
        [-0.9786, -0.4713],
        [-1.1452, -0.3830],
        [-0.9982, -0.4597],
        [-1.2678, -0.3305],
        [-1.0664, -0.4220],
        [-0.9494, -0.4893],
        [-1.2222, -0.3490],
        [-1.6441, -0.2147],
        [-1.0983, -0.4056],
        [-1.3653, -0.2948],
        [-1.1547, -0.3785],
        [-1.0880, -0.4108],
        [-1.6052, -0.2242],
        [-0.7709, -0.6210],
        [-1.5943, -0.2270]], grad_fn=<LogSoftmaxBackward0>)

In [72]:
d_model = 128
n_head = 8
num_layers = 2
dropout = 0.3

embedding = nn.Sequential(
    nn.Linear(14, d_model),
    nn.ReLU(),
)
conv = nn.Conv1d(d_model, d_model, kernel_size=5, padding=2)
pe = nn.Parameter(torch.randn(1, d_model))
transformer = nn.Transformer(d_model, n_head, num_layers, dropout=dropout)
fc = nn.Linear(d_model, 2)

output = conv(embedding(x).permute(0, 2, 1)).permute(0, 2, 1)
output = output + pe
output = output.permute(1, 0, 2)
out = transformer.encoder(output)
out = out[-1, :, :]
out = fc(out)
out.size()

torch.Size([32, 2])

In [73]:
out

tensor([[-0.1880,  0.2474],
        [ 0.4479, -0.0553],
        [ 0.7414,  0.5551],
        [ 0.2159,  0.9847],
        [ 0.3612,  0.9007],
        [-0.7108, -0.0406],
        [ 0.6071, -0.4828],
        [ 0.3353,  0.6837],
        [-0.1599, -0.3562],
        [ 0.9408,  0.7332],
        [ 0.2811,  0.0217],
        [ 0.3657,  0.4245],
        [ 0.0273, -0.1086],
        [-0.1108,  0.1441],
        [ 0.5720,  0.3567],
        [ 0.4527,  0.3909],
        [ 0.5528,  0.6263],
        [ 0.3453,  0.4314],
        [ 0.2753, -0.2350],
        [-0.0679,  1.1366],
        [ 0.7679,  0.5170],
        [ 0.4683,  0.7359],
        [ 0.0187, -0.1772],
        [ 0.8656,  0.8362],
        [ 0.0331,  0.2159],
        [ 0.6308,  0.4083],
        [ 0.1872,  0.7556],
        [ 0.7465,  0.2256],
        [ 0.2023,  0.7617],
        [ 0.5142,  0.2068],
        [ 0.4845,  0.9135],
        [ 0.3157,  0.4350]], grad_fn=<AddmmBackward0>)

In [3]:
from pathlib import Path
data_path = Path("/media/hdd1/i-care/6h-combined/0284/12_18_features.npy")
import numpy as np
np.load(data_path).shape

(296, 144)